In [2]:
# Single Jupyter cell: Full preprocessing + IG/Chi2 + CUCKOO SEARCH (SMOTE-in-CV)
# ---------------------------------------------------------------------------
import os, time
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

from IPython.display import display
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

# -------------------------
# USER CONFIG - adjust paths / mode if needed
# -------------------------
DATA_PATH = r"C:\Users\haris\OneDrive\Documents\FINAL YEAR PROJECT\dataset\OneDrive_2025-06-05 (1)\CSV Files\Training and Testing Sets"
TRAIN_PATH = os.path.join(DATA_PATH, "UNSW_train_new.csv")
TEST_PATH  = os.path.join(DATA_PATH, "UNSW_test_new.csv")

MODE = "binary"   # "binary" or "multiclass"
USE_MULTICLASS = True if MODE == "multiclass" else False

SAVE_DIR = "feature_lists"
os.makedirs(SAVE_DIR, exist_ok=True)

# Cuckoo params (keep small for debug; increase for full runs)
CUCKOO_NESTS = 25
CUCKOO_ITERS = 1

# CV for metaheuristic fitness
CV_SPLITS = 3

# -------------------------
# Helpers: load + preprocess functions
# -------------------------
def load_data(train_path, test_path):
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)
    print("Loaded train:", df_train.shape, "test:", df_test.shape)
    return df_train, df_test

def preprocess_fit(df, mode="binary"):
    """
    Fit encoders & scaler on training df and return processed df + scaler + encoders + target col.
    Also DROP the "other" target column to avoid leakage.
    """
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower()

    # Determine target
    if mode == "multiclass":
        target = 'attack_cat' if 'attack_cat' in df.columns else 'label'
    else:
        # ensure label exists from attack_cat if needed
        if 'label' not in df.columns and 'attack_cat' in df.columns:
            df['label'] = df['attack_cat'].apply(lambda x: 0 if str(x).strip().lower() in ['normal','normal.'] else 1)
        target = 'label'

    # Drop the other target to avoid leakage
    if target == 'label' and 'attack_cat' in df.columns:
        df.drop(columns=['attack_cat'], inplace=True, errors='ignore')
        print("Dropped 'attack_cat' from training features (binary mode).")
    if target == 'attack_cat' and 'label' in df.columns:
        df.drop(columns=['label'], inplace=True, errors='ignore')
        print("Dropped 'label' from training features (multiclass mode).")

    # Drop obvious identifiers
    for c in ['id','srcip','dstip']:
        if c in df.columns:
            df.drop(columns=[c], inplace=True, errors='ignore')

    # Numeric fillna by median
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    if numeric_cols:
        df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

    # Categorical fillna then LabelEncode (store encoders)
    df = df.fillna(df.mode().iloc[0])
    encoders = {}
    for col in df.select_dtypes(include=['object']).columns:
        if col == target:
            continue
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        encoders[col] = le

    # Scale features (fit scaler)
    feature_cols = [c for c in df.columns if c != target]
    scaler = MinMaxScaler()
    if feature_cols:
        df[feature_cols] = scaler.fit_transform(df[feature_cols])

    return df, scaler, encoders, target

def preprocess_apply(df, scaler, encoders, target, mode="binary"):
    """
    Apply encoders & scaler to test set. Drop the non-target column to avoid leakage.
    """
    df = df.copy()
    df.columns = df.columns.str.strip().str.lower()

    # If test uses attack_cat but our chosen target is 'label', ensure label exists
    if target == 'label' and 'label' not in df.columns and 'attack_cat' in df.columns:
        df['label'] = df['attack_cat'].apply(lambda x: 0 if str(x).strip().lower() in ['normal','normal.'] else 1)

    # Drop the other target column in test set too
    if target == 'label' and 'attack_cat' in df.columns:
        df.drop(columns=['attack_cat'], inplace=True, errors='ignore')
        print("Dropped 'attack_cat' from test features (binary mode).")
    if target == 'attack_cat' and 'label' in df.columns:
        df.drop(columns=['label'], inplace=True, errors='ignore')
        print("Dropped 'label' from test features (multiclass mode).")

    # Drop identifiers
    for c in ['id','srcip','dstip']:
        if c in df.columns:
            df.drop(columns=[c], inplace=True, errors='ignore')

    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    if numeric_cols:
        df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
    df = df.fillna(df.mode().iloc[0])

    # Apply stored encoders safely
    for col, le in encoders.items():
        if col in df.columns:
            try:
                df[col] = le.transform(df[col].astype(str))
            except Exception:
                mapping = {c: i for i, c in enumerate(le.classes_)}
                unk = len(le.classes_)
                df[col] = [mapping.get(str(v), unk) for v in df[col].astype(str)]

    # Scale using provided scaler
    feature_cols = [c for c in df.columns if c != target]
    if feature_cols:
        df[feature_cols] = scaler.transform(df[feature_cols])

    return df

# -------------------------
# Load & preprocess
# -------------------------
print("Loading data...")
df_train_raw, df_test_raw = load_data(TRAIN_PATH, TEST_PATH)

df_train_proc, scaler, encoders, target_col = preprocess_fit(df_train_raw, mode=MODE)
df_test_proc  = preprocess_apply(df_test_raw, scaler, encoders, target_col, mode=MODE)

# If multiclass ensure integer mapping for attack_cat consistently (LabelEncode target)
le_target = None
if MODE == "multiclass" and target_col == 'attack_cat':
    le_target = LabelEncoder()
    df_train_proc['attack_cat'] = le_target.fit_transform(df_train_proc['attack_cat'].astype(str))
    # apply mapping to test (if possible)
    df_test_proc['attack_cat'] = le_target.transform(df_test_proc['attack_cat'].astype(str))

print("Preprocessing done. Train shape:", df_train_proc.shape, "Test shape:", df_test_proc.shape)
print("Train target distribution:\n", df_train_proc[target_col].value_counts())
print("Test target distribution:\n", df_test_proc[target_col].value_counts())

# Optional duplicate row check to ensure no leakage
def duplicate_row_check(df_train_raw, df_test_raw):
    drop_cols = [c for c in ['id','srcip','dstip'] if c in df_train_raw.columns]
    tr = df_train_raw.drop(columns=drop_cols, errors='ignore').astype(str)
    te = df_test_raw.drop(columns=drop_cols, errors='ignore').astype(str)
    import hashlib
    def row_hashes(df):
        return df.apply(lambda row: hashlib.md5('|'.join(row.values).encode()).hexdigest(), axis=1)
    h_tr = set(row_hashes(tr))
    h_te = set(row_hashes(te))
    dups = len(h_tr.intersection(h_te))
    print("Exact duplicate rows between train & test (by hash):", dups)

duplicate_row_check(df_train_raw, df_test_raw)

# -------------------------
# IG + Chi2 on original training (no SMOTE)
# -------------------------
X_train_base = df_train_proc.drop(columns=[target_col])
y_train_base = df_train_proc[target_col].astype(int)

print("\nComputing IG and Chi2 on ORIGINAL training (no SMOTE)...")
ig_scores = mutual_info_classif(X_train_base.values, y_train_base.values, random_state=42)
chi2_scores, _ = chi2(X_train_base.values, y_train_base.values)

ig_df = pd.DataFrame({'feature': X_train_base.columns, 'ig': ig_scores}).sort_values('ig', ascending=False)
chi_df = pd.DataFrame({'feature': X_train_base.columns, 'chi2': chi2_scores}).sort_values('chi2', ascending=False)

display(ig_df.head(10))
display(chi_df.head(10))

# choose threshold (median) and union
ig_thresh = np.median(ig_df['ig'])
chi_thresh = np.median(chi_df['chi2'])
ig_selected = set(ig_df[ig_df['ig'] >= ig_thresh]['feature'].tolist())
chi_selected = set(chi_df[chi_df['chi2'] >= chi_thresh]['feature'].tolist())
combined_features = sorted(list(ig_selected.union(chi_selected)))

print(f"IG selected {len(ig_selected)}; Chi2 selected {len(chi_selected)}; Combined union = {len(combined_features)}")
pd.Series(sorted(list(ig_selected))).to_csv(os.path.join(SAVE_DIR, "ig_selected.csv"), index=False)
pd.Series(sorted(list(chi_selected))).to_csv(os.path.join(SAVE_DIR, "chi2_selected.csv"), index=False)
pd.Series(combined_features).to_csv(os.path.join(SAVE_DIR, "combined_ig_chi2.csv"), index=False)

# align combined features with test set (names only)
common_feats = [f for f in combined_features if f in df_test_proc.columns]
if len(common_feats) != len(combined_features):
    missing = set(combined_features) - set(common_feats)
    print("Warning: these features not in test set and will be dropped:", missing)
combined_features = common_feats
print("Using combined_features count after alignment:", len(combined_features))

# arrays used by metaheuristics (they'll use SMOTE inside CV)
X_comb_train = X_train_base[combined_features].values if len(combined_features) > 0 else np.empty((X_train_base.shape[0],0))
y_comb_train = y_train_base.values

# -------------------------
# Metaheuristic: Cuckoo Search (binary mask) with SMOTE-in-CV fitness
# -------------------------
cv = StratifiedKFold(n_splits=CV_SPLITS, shuffle=True, random_state=42)

def make_smote_rf_pipeline():
    # small RF inside CV for fitness; keep n_jobs=1 to avoid nested parallelism issues
    return ImbPipeline([
        ('smote', SMOTE(random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=1))
    ])

def cuckoo_search_binary(X, y, feature_names, n_nests=CUCKOO_NESTS, max_iter=CUCKOO_ITERS, random_state=42):
    """
    Binary cuckoo: nests are binary masks over features. Fitness uses cross_val_score with SMOTE+RF pipeline.
    """
    np.random.seed(random_state)
    n_features = X.shape[1]
    if n_features == 0:
        return np.array([], dtype=bool), [], 0.0

    # initialize nests (binary masks)
    nests = np.random.randint(0,2,size=(n_nests, n_features)).astype(float)
    fitness_cache = {}
    pipe = make_smote_rf_pipeline()

    def fitness(mask):
        key = tuple(mask.astype(int).tolist())
        if key in fitness_cache:
            return fitness_cache[key]
        if mask.sum() == 0:
            fitness_cache[key] = 0.0
            return 0.0
        sel = mask.astype(bool)
        X_sel = X[:, sel]
        try:
            scores = cross_val_score(pipe, X_sel, y, cv=cv, scoring='accuracy', n_jobs=1)
            fitness_cache[key] = scores.mean()
        except Exception as e:
            # If CV fails (e.g., too-few samples for some classes), penalize
            fitness_cache[key] = 0.0
        return fitness_cache[key]

    fitness_vals = np.array([fitness(nests[i]) for i in range(n_nests)])
    best_overall = fitness_vals.max()
    for it in range(max_iter):
        # For each nest attempt a single-bit flip (simple local search) and accept if better
        for i in range(n_nests):
            j = np.random.randint(0, n_features)
            trial = nests[i].copy()
            trial[j] = 1 - trial[j]
            f_trial = fitness(trial)
            if f_trial > fitness_vals[i]:
                nests[i] = trial
                fitness_vals[i] = f_trial
        # Abandon a fraction of worst nests -> reinitialize
        worst_idx = np.argsort(fitness_vals)[:max(1, n_nests//5)]
        for idx in worst_idx:
            nests[idx] = np.random.randint(0,2,size=n_features).astype(float)
            fitness_vals[idx] = fitness(nests[idx])
        best_overall = fitness_vals.max()
        if (it+1) % max(1, max_iter//5) == 0 or it == max_iter-1:
            print(f"Cuckoo iter {it+1}/{max_iter} -- best CV acc = {best_overall:.4f}")

    best_idx = np.argmax(fitness_vals)
    best_mask = nests[best_idx].astype(bool)
    selected = [feature_names[i] for i,m in enumerate(best_mask) if m]
    best_score = fitness_vals[best_idx]
    return best_mask, selected, best_score

# -------------------------
# Run cuckoo only
# -------------------------
if len(combined_features) == 0:
    print("No combined features available for Cuckoo Search. Exiting.")
else:
    print("\nRunning Cuckoo Search (ONLY)...")
    t0 = time.time()
    cuckoo_mask, cuckoo_selected_features, cuckoo_score = cuckoo_search_binary(
        X_comb_train, y_comb_train, combined_features,
        n_nests=CUCKOO_NESTS, max_iter=CUCKOO_ITERS, random_state=42
    )
    t_cuckoo = time.time() - t0
    print(f"\nCuckoo selected {len(cuckoo_selected_features)} features; CV-estimated accuracy: {cuckoo_score:.4f}; time {t_cuckoo:.1f}s")
    # Save selection
    pd.Series(sorted(cuckoo_selected_features)).to_csv(os.path.join(SAVE_DIR, "cuckoo_selected.csv"), index=False)
    print("Saved cuckoo_selected.csv to", os.path.join(SAVE_DIR, "cuckoo_selected.csv"))
    # show selected features
    display(pd.Series(cuckoo_selected_features).to_frame("selected_features").reset_index(drop=True))

print("CUCKOO step complete. Next: load 'feature_lists/cuckoo_selected.csv' in your SHAP/LIME notebook cell.")



import pickle
import os

# -------------------------------------------
# Where to save files (same folder used in app.py)
# -------------------------------------------
SAVE_DIR = "model"
os.makedirs(SAVE_DIR, exist_ok=True)

# -------------------------------------------
# REQUIRED VARIABLES (already exist in your notebook)
# -------------------------------------------
# model: your final trained classifier
# scaler: MinMaxScaler object used in preprocessing
# encoders: dict of LabelEncoder objects (for categorical features)
# combined_features OR cuckoo_selected_features: final features selected
# le_target: LabelEncoder for attack_cat (only if multiclass)

# -------------------------------------------
# 1. Save Model
# -------------------------------------------
pickle.dump(model, open(os.path.join(SAVE_DIR, "xgb_model.pkl"), "wb"))
print("Saved: model/xgb_model.pkl")

# -------------------------------------------
# 2. Save Scaler
# -------------------------------------------
pickle.dump(scaler, open(os.path.join(SAVE_DIR, "scaler.pkl"), "wb"))
print("Saved: model/scaler.pkl")

# -------------------------------------------
# 3. Save Encoders Dictionary
# -------------------------------------------
pickle.dump(encoders, open(os.path.join(SAVE_DIR, "encoders.pkl"), "wb"))
print("Saved: model/encoders.pkl")

# -------------------------------------------
# 4. Save Selected Feature List
# -------------------------------------------
# Use your final selected features from Cuckoo Search
if 'cuckoo_selected_features' in globals():
    final_features = cuckoo_selected_features
elif 'combined_features' in globals():
    final_features = combined_features
else:
    raise Exception("No feature list found! Provide selected features.")

pickle.dump(final_features, open(os.path.join(SAVE_DIR, "feature_list.pkl"), "wb"))
print("Saved: model/feature_list.pkl")

# -------------------------------------------
# 5. Save attack_cat label encoder (multiclass mode)
# -------------------------------------------
if MODE == "multiclass" and le_target is not None:
    pickle.dump(le_target, open(os.path.join(SAVE_DIR, "target_encoder.pkl"), "wb"))
    print("Saved: model/target_encoder.pkl")
else:
    print("Binary mode – no target encoder needed.")


Loading data...
Loaded train: (206138, 45) test: (51535, 45)
Dropped 'attack_cat' from training features (binary mode).
Dropped 'attack_cat' from test features (binary mode).
Preprocessing done. Train shape: (206138, 43) Test shape: (51535, 43)
Train target distribution:
 label
1    131738
0     74400
Name: count, dtype: int64
Test target distribution:
 label
1    32935
0    18600
Name: count, dtype: int64
Exact duplicate rows between train & test (by hash): 10425

Computing IG and Chi2 on ORIGINAL training (no SMOTE)...


feature        ig
6         sbytes  0.455177
7         dbytes  0.337735
11         sload  0.337056
31  ct_state_ttl  0.334323
26         smean  0.334314
9           sttl  0.332182
10          dttl  0.312662
8           rate  0.311163
0            dur  0.300306
27         dmean  0.280903

feature          chi2
9               sttl  18362.972813
19              swin  14246.026783
22              dwin  12544.049945
12             dload  10095.261168
34  ct_dst_sport_ltm   7093.126797
8               rate   6529.910533
27             dmean   6318.303660
20             stcpb   6296.733436
21             dtcpb   6216.770053
31      ct_state_ttl   5788.814582

IG selected 21; Chi2 selected 21; Combined union = 34
Using combined_features count after alignment: 34

Running Cuckoo Search (ONLY)...
Cuckoo iter 1/1 -- best CV acc = 0.9468

Cuckoo selected 20 features; CV-estimated accuracy: 0.9468; time 3089.6s
Saved cuckoo_selected.csv to feature_lists\cuckoo_selected.csv


selected_features
0         ct_dst_ltm
1     ct_dst_src_ltm
2   ct_src_dport_ltm
3         ct_srv_src
4             dbytes
5               djit
6              dload
7              dmean
8              dpkts
9              dtcpb
10              dttl
11              rate
12            sbytes
13           service
14            sinpkt
15             smean
16             state
17             stcpb
18              swin
19            tcprtt

CUCKOO step complete. Next: load 'feature_lists/cuckoo_selected.csv' in your SHAP/LIME notebook cell.


NameError: name 'model' is not defined

In [5]:
# ================================
# CELL 2 — FINAL TRAINING + SCALER REFIT (CORRECT)
# ================================
import pickle
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os

# ---------------------------
# 1. Determine final selected features
# ---------------------------
if "cuckoo_selected_features" in globals() and len(cuckoo_selected_features) > 0:
    final_features = sorted(list(cuckoo_selected_features))
    print("Using Cuckoo-selected features.")
elif "combined_features" in globals() and len(combined_features) > 0:
    final_features = sorted(list(combined_features))
    print("Using Combined IG+Chi2 features.")
else:
    raise Exception("ERROR: No feature list found!")

print("\nFinal feature count:", len(final_features))
print("Final features:", final_features)

# ---------------------------
# 2. REFIT SCALER ON FINAL FEATURES (REQUIRED!)
# ---------------------------
scaler = MinMaxScaler()
X_train_final = df_train_proc[final_features]
X_train_scaled = scaler.fit_transform(X_train_final)

y_train_final = df_train_proc[target_col]

print("\nScaler refit complete. Shape:", X_train_scaled.shape)

# ---------------------------
# 3. Train FINAL MODEL
# ---------------------------
model = XGBClassifier(
    n_estimators=300,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="multi:softprob" if MODE=="multiclass" else "binary:logistic",
    eval_metric="mlogloss" if MODE=="multiclass" else "logloss",
    random_state=42
)

model.fit(X_train_scaled, y_train_final)

print("\n🎉 Final XGBoost model trained successfully!")
print("Model expects:", model.n_features_in_)

# ---------------------------
# 4. SAVE FILES FOR FLASK
# ---------------------------
SAVE_DIR = "model"
os.makedirs(SAVE_DIR, exist_ok=True)

pickle.dump(model, open(os.path.join(SAVE_DIR, "xgb_model.pkl"), "wb"))
print("Saved model/xgb_model.pkl")

pickle.dump(scaler, open(os.path.join(SAVE_DIR, "scaler.pkl"), "wb"))
print("Saved model/scaler.pkl")

pickle.dump(encoders, open(os.path.join(SAVE_DIR, "encoders.pkl"), "wb"))
print("Saved model/encoders.pkl")

pickle.dump(final_features, open(os.path.join(SAVE_DIR, "feature_list.pkl"), "wb"))
print("Saved model/feature_list.pkl")

if MODE == "multiclass" and "le_target" in globals():
    pickle.dump(le_target, open(os.path.join(SAVE_DIR, "target_encoder.pkl"), "wb"))
    print("Saved model/target_encoder.pkl")
else:
    print("Binary mode — no target encoder.")

print("\n✔ All model files saved successfully! Flask will load correct versions.")


Using Cuckoo-selected features.

Final feature count: 20
Final features: ['ct_dst_ltm', 'ct_dst_src_ltm', 'ct_src_dport_ltm', 'ct_srv_src', 'dbytes', 'djit', 'dload', 'dmean', 'dpkts', 'dtcpb', 'dttl', 'rate', 'sbytes', 'service', 'sinpkt', 'smean', 'state', 'stcpb', 'swin', 'tcprtt']

Scaler refit complete. Shape: (206138, 20)

🎉 Final XGBoost model trained successfully!
Model expects: 20
Saved model/xgb_model.pkl
Saved model/scaler.pkl
Saved model/encoders.pkl
Saved model/feature_list.pkl
Binary mode — no target encoder.

✔ All model files saved successfully! Flask will load correct versions.


In [6]:
import pickle

# Load feature list
features = pickle.load(open("model/feature_list.pkl", "rb"))

print("\n===== FINAL FEATURE LIST (MODEL INPUT ORDER) =====")
for f in features:
    print(f)

print("\nTotal features:", len(features))



===== FINAL FEATURE LIST (MODEL INPUT ORDER) =====
ct_dst_ltm
ct_dst_src_ltm
ct_src_dport_ltm
ct_srv_src
dbytes
djit
dload
dmean
dpkts
dtcpb
dttl
rate
sbytes
service
sinpkt
smean
state
stcpb
swin
tcprtt

Total features: 20


In [7]:
scaler = pickle.load(open("model/scaler.pkl", "rb"))
print("\nScaler feature names:", getattr(scaler, "feature_names_in_", None))



Scaler feature names: ['ct_dst_ltm' 'ct_dst_src_ltm' 'ct_src_dport_ltm' 'ct_srv_src' 'dbytes'
 'djit' 'dload' 'dmean' 'dpkts' 'dtcpb' 'dttl' 'rate' 'sbytes' 'service'
 'sinpkt' 'smean' 'state' 'stcpb' 'swin' 'tcprtt']


In [18]:
# ================================
# CELL 3 — FIXED SAVE ARTIFACTS
# ================================
import pickle
import os

SAVE_DIR = "model"
os.makedirs(SAVE_DIR, exist_ok=True)

# Save model
pickle.dump(model, open(f"{SAVE_DIR}/xgb_model.pkl", "wb"))
print("Saved model/xgb_model.pkl")

# Save scaler (already refit for 20 features)
pickle.dump(scaler, open(f"{SAVE_DIR}/scaler.pkl", "wb"))
print("Saved model/scaler.pkl")

# Save encoders (no changes)
pickle.dump(encoders, open(f"{SAVE_DIR}/encoders.pkl", "wb"))
print("Saved model/encoders.pkl")

# Save FINAL feature list (matching EXACTLY the trained model)
pickle.dump(final_features, open(f"{SAVE_DIR}/feature_list.pkl", "wb"))
print("Saved model/feature_list.pkl")

# Save target encoder (only for multiclass)
if MODE == "multiclass" and 'le_target' in globals() and le_target is not None:
    pickle.dump(le_target, open(f"{SAVE_DIR}/target_encoder.pkl", "wb"))
    print("Saved model/target_encoder.pkl")
else:
    print("Binary mode – no target encoder needed.")

print("\n🎉 All artifacts saved successfully!")


Saved model/xgb_model.pkl
Saved model/scaler.pkl
Saved model/encoders.pkl
Saved model/feature_list.pkl
Binary mode – no target encoder needed.

🎉 All artifacts saved successfully!


In [19]:
print("Scaler expects:", scaler.n_features_in_)
print("Model expects:", model.n_features_in_)

loaded_features = pickle.load(open("model/feature_list.pkl", "rb"))
print("Feature list count:", len(loaded_features))


Scaler expects: 20
Model expects: 20
Feature list count: 20


In [11]:
print("Service categories:", encoders["service"].classes_)


Service categories: ['-' 'dhcp' 'dns' 'ftp' 'ftp-data' 'http' 'irc' 'pop3' 'radius' 'smtp'
 'snmp' 'ssh' 'ssl']


In [7]:
import pandas as pd

feat = pd.read_csv("feature_lists/cuckoo_selected.csv", header=None)[0].tolist()
print(feat)


['0', 'ct_dst_ltm', 'ct_dst_src_ltm', 'ct_src_dport_ltm', 'ct_srv_src', 'dbytes', 'djit', 'dload', 'dmean', 'dpkts', 'dtcpb', 'dttl', 'rate', 'sbytes', 'service', 'sinpkt', 'smean', 'state', 'stcpb', 'swin', 'tcprtt']


In [12]:
import pickle
import os

SAVE_DIR = "model"
os.makedirs(SAVE_DIR, exist_ok=True)

# MUST match the exact order used during training
final_features = cuckoo_selected_features   # or your final feature list variable

# Filter your X_train to only these 20 features BEFORE training
# model.fit(X_train[final_features], y_train)

pickle.dump(model, open(f"{SAVE_DIR}/xgb_model.pkl", "wb"))
pickle.dump(scaler, open(f"{SAVE_DIR}/scaler.pkl", "wb"))
pickle.dump(encoders, open(f"{SAVE_DIR}/encoders.pkl", "wb"))
pickle.dump(final_features, open(f"{SAVE_DIR}/feature_list.pkl", "wb"))

# Save target label encoder (only if multiclass)
if MODE == "multiclass":
    pickle.dump(le_target, open(f"{SAVE_DIR}/target_encoder.pkl", "wb"))


In [13]:
print("Model expects:", model.n_features_in_)
print("Scaler expects:", scaler.n_features_in_)
print("Feature list length:", len(final_features))


Model expects: 20
Scaler expects: 42
Feature list length: 20


In [2]:
import pickle
import numpy as np

# take 500 random samples as LIME background (recommended)
idx = np.random.choice(len(Xs_train), size=min(500, len(Xs_train)), replace=False)
lime_bg_unsw = Xs_train[idx]

# save background
with open("model/lime_background_unsw.pkl", "wb") as f:
    pickle.dump(lime_bg_unsw, f)

print("UNSW LIME background saved!")


NameError: name 'Xs_train' is not defined

In [4]:
import pickle
import pandas as pd
import numpy as np

# ---------------- LOAD ARTIFACTS ----------------
UNSW_FEAT = pickle.load(open("model/feature_list.pkl", "rb"))
UNSW_FEAT = [f.lower() for f in UNSW_FEAT]

UNSW_SCALER = pickle.load(open("model/scaler.pkl", "rb"))
UNSW_ENCODERS = pickle.load(open("model/encoders.pkl", "rb"))

# ---------------- LOAD TRAINING CSV ----------------
# update path if needed
df = pd.read_csv(r"C:\Users\haris\OneDrive\Documents\FINAL YEAR PROJECT\dataset\OneDrive_2025-06-05 (1)\CSV Files\Training and Testing Sets\UNSW_train_new.csv")

# preprocessing same as unsw_prepare() ----------------
df.columns = df.columns.str.lower().str.strip()

# add missing features
for c in UNSW_FEAT:
    if c not in df.columns:
        df[c] = 0

df = df[UNSW_FEAT].copy()

# encode categoricals
for c in df.columns:
    if c in UNSW_ENCODERS:
        le = UNSW_ENCODERS[c]
        classes = list(le.classes_)
        mapping = {v:i for i,v in enumerate(classes)}
        unk = len(classes)
        df[c] = df[c].astype(str).map(mapping).fillna(unk).astype(int)

# numeric columns
for c in df.columns:
    if c not in UNSW_ENCODERS:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

# scale
Xs = UNSW_SCALER.transform(df.astype(float))

# ---------------- SELECT BACKGROUND ----------------
bg_size = min(500, len(Xs))
idx = np.random.choice(len(Xs), size=bg_size, replace=False)
lime_bg_unsw = Xs[idx]

# ---------------- SAVE BACKGROUND ----------------
with open("model/lime_background_unsw.pkl", "wb") as f:
    pickle.dump(lime_bg_unsw, f)

print("✅ UNSW LIME background saved:", lime_bg_unsw.shape)


✅ UNSW LIME background saved: (500, 20)


In [5]:
UNSW_LIME_BG = pickle.load(open(UNSW_LIME_BG_PATH, "rb"))


NameError: name 'UNSW_LIME_BG_PATH' is not defined